# Test the endpoint

In [17]:
import json, os, sys
import pandas as pd
import numpy as np

In [18]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.huggingface.model import HuggingFaceModel
from sagemaker.workflow.model_step import ModelStep
from sagemaker.workflow.pipeline_context import PipelineSession

from src.utils import config

In [19]:
client = boto3.client('sagemaker-runtime')

session = sagemaker.Session()
default_bucket = session.default_bucket()

iam = boto3.client('iam')
role_arn = iam.get_role(RoleName=f'101436505502-sagemaker-exec')['Role']['Arn']

# load local csv
df = pd.read_csv("data/mtsamples.csv")
df = df[df["transcription"].notna()]

## Run Inference on deployed endpoint

In [20]:
# get random sample
idx = np.random.randint(len(df))
input_txt = df.loc[idx].transcription
category = df.loc[idx].medical_specialty

print(category)
print(input_txt)

 Radiology
EXAM:,MRI LEFT SHOULDER,CLINICAL:,This is a 69-year-old male with pain in the shoulder. Evaluate for rotator cuff tear.,FINDINGS:,Examination was performed on 9/1/05.,There is marked supraspinatus tendinosis and extensive tearing of the substance of the tendon and articular surface, extending into the myotendinous junction as well. There is still a small rim of tendon along the bursal surface, although there may be a small tear at the level of the rotator interval. There is no retracted tendon or muscular atrophy (series #6 images #6-17).,Normal infraspinatus tendon.,There is subscapularis tendinosis with fraying and partial tearing of the superior most fibers extending to the level of the rotator interval (series #9 images #8-13; series #3 images #8-14). There is no complete tear, gap or fiber retraction and there is no muscular atrophy.,There is tendinosis and superficial tearing of the long biceps tendon within the bicipital groove, and there is high grade (near complete)

In [34]:
endpoint_name = "53-2023-04-12-13-25-22-053"
content_type = "application/json" #
accept = "application/json" #"text/csv"
payload = json.dumps({"instances":[input_txt]})

response = client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType=content_type,
    Accept=accept,
    Body=payload,
)

json.loads(response["Body"].read())

{'prediction': [' Orthopedic']}

# Register model (for testing model inference code)
Use model data from trained model

In [22]:
# gpu_instance_type = "ml.g4dn.xlarge"
# pytorch_version = "1.9.0"
# transformers_version = "4.11.0"
# py_version = "py38"

# requirement_dependencies = [os.path.join(
#     # sys.path[0], 
#     "images", "inference", "requirements.txt")]

# model = HuggingFaceModel(
#     name="text-classification-model-123",
#     model_data="s3://sagemaker-eu-west-3-101436505502/model/pipelines-kam521dashky-train-model-JzSCBl76CF/output/model.tar.gz",
#     sagemaker_session=session,
#     source_dir="src",
#     entry_point="model.py",
#     dependencies=['requirements.txt'],
#     role=role_arn,
#     transformers_version=transformers_version,
#     pytorch_version=pytorch_version,
#     py_version=py_version,
# )

## Run batch inference (Transformer)
Use latest approved model from model registry

In [26]:
def get_latest_approved_model(model_package_group_name):
    sm_client = boto3.client('sagemaker')
    df = pd.DataFrame(sm_client.list_model_packages(
        ModelPackageGroupName=model_package_group_name)["ModelPackageSummaryList"])
    model_package_arn = df.loc[df.ModelApprovalStatus == "Approved"].iloc[0].ModelPackageArn
    print(f"The latest approved model-arn is: {model_package_arn}")
    return sagemaker.ModelPackage(role=role_arn, model_package_arn=model_package_arn, sagemaker_session=session)


def run_batch_inference(input_data_path, output_data_path):
    CONTENT_TYPE_CSV = 'text/csv'
    CONTENT_TYPE_JSON = "application/json"

    model = get_latest_approved_model("training-pipelineModelGroup")
    transformer =model.transformer(
        instance_count = 1,
        instance_type = 'ml.g4dn.xlarge', #"ml.m5.large"
        output_path = output_data_path,
        accept = CONTENT_TYPE_CSV,
        # strategy = 'SingleRecord',
        # assemble_with = 'Line',
    )
    
    transformer.transform(
        data=input_data_path,
        content_type=CONTENT_TYPE_CSV,
        # split_type='Line',
    )
    transformer.wait()

In [29]:
input_data_path = f"s3://{default_bucket}/data/test.csv"
output_data_path = f"s3://{default_bucket}/data/out"

run_batch_inference(input_data_path, output_data_path)

arn:aws:sagemaker:eu-west-3:101436505502:model-package/training-pipelinemodelgroup/53


INFO:sagemaker:Creating model with name: 53-2023-04-12-12-35-31-941


# Torch test area

In [27]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm


class MyTokenizer:
    def __init__(self, model_name="distilbert-base-uncased") -> None:
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)

    def tokenize(self, txt_input):
        return self.tokenizer.encode(txt_input, padding="max_length", truncation=True)
    
    
model = AutoModelForSequenceClassification.from_pretrained(
        "distilbert-base-uncased",
        num_labels=40,
    )


class MyDataset(Dataset):
    def __init__(self, x, y) -> None:
        self.x = torch.tensor(x)
        self.y = torch.tensor(y)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classi

In [17]:
tok = MyTokenizer()
texts = list(df.transcription.values)

inputs = tok.tokenizer(texts, padding="max_length", return_tensors='pt', truncation=True)
inputs

{'input_ids': tensor([[  101, 20714,  1024,  ...,     0,     0,     0],
        [  101,  2627,  2966,  ...,  1010, 26572,   102],
        [  101,  2381,  1997,  ...,  4645,  2545,   102],
        ...,
        [  101, 20714,  1024,  ...,  8048,  1999,   102],
        [  101,  2708, 12087,  ..., 27179,  1024,   102],
        [  101,  2381,  1024,  ..., 17758,  2135,   102]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])}

In [22]:
dataset = MyDataset(inputs.input_ids, inputs.attention_mask)

dataloader = DataLoader(dataset, shuffle=False, batch_size=10)


/var/folders/b8/mx2blhp93k7blwkppkkv1w_r0000gn/T/ipykernel_9997/2213946952.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.x = torch.tensor(x)
/var/folders/b8/mx2blhp93k7blwkppkkv1w_r0000gn/T/ipykernel_9997/2213946952.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y = torch.tensor(y)


In [32]:
output = []
for (x, _) in tqdm(list(enumerate(dataloader))[:4]):
    outs = model(x)
    output += torch.argmax(outs.logits.cpu(), dim=1)

print(output)

  0%|          | 0/4 [00:00<?, ?it/s]


AttributeError: 'int' object has no attribute 'size'